In [ ]:
#%matplotlib inline
%matplotlib notebook

import glob
import matplotlib.pylab as plt
import numpy as np
import os
import struct
import warnings

from __future__ import print_function
from matplotlib.mlab import detrend
from matplotlib import dates
from obspy import *
from obspy.core import read, UTCDateTime
from obspy.clients.fdsn import Client
from obspy.signal.cross_correlation import xcorr_pick_correction
from obspy.signal.filter import lowpass
from obspy.signal.invsim import cosine_taper
from scipy import interpolate, signal
from time import *

from lib_seis import markp_min, wf_fft

warnings.filterwarnings('ignore')                  # do not show warnings

plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = 8, 6
plt.rcParams['lines.linewidth'] = 1

# Adaptation of CAN_response.m by Carl Tape
# Initial coding by: Yongki Aiman, Tomy Gunawan, Angel Ling
# Coding by Amanda McPherson, 2020
# The example waveform is from CAN (Canberra, Australia) for the 2004 Mw 9.X Sumatra-Andaman earthquake.
# custom functions: wf_fft

In [ ]:
# retrieve the waveform
client = Client("IRIS")
t = UTCDateTime("2004-12-26T00:58:53.0")
starttime = t-(0.5*24*3600) # 0.5 dayS before the earthquake
endtime   = t+(9.0*24*3600) # 9 days after the earthquake

st = client.get_waveforms("G", "CAN", "*", "LHZ", starttime, endtime, attach_response=True)
print(st)

In [ ]:
# take a copy of the stream to avoid overwriting the original data
can = st.copy()

# choose the 1st stream; specify sampling parameters and Nyquist frequency
npts = can[0].stats.npts                 # number of samples
df = can[0].stats.sampling_rate          # sampling rate
nsec = npts/df                           # sampling time
fNy = df / 2.0                           # Nyquist frequency
time = np.linspace(0,nsec,int(nsec*df))  # time axis for plotting in seconds
date = can[0].times(type='utcdatetime') # time axis for plotting absolute time
date = dates.date2num(date)

y  = can[0].data                      # set up signal 

# Plot the result with time in seconds
plt.figure()
plt.plot(time, y, 'b') 
# FIX: stations/channel/time should not be hardcoded
plt.title('CAN (LHZ) − starting 2004−12−25 12:36:12.656')
plt.xlabel('Time [s]')
plt.ylabel('Counts')

# Plot the result with absolute time
plt.figure()
ax = plt.gca()
plt.plot_date(date,y,'b')
plt.title('CAN (LHZ) − starting 2004−12−25 12:36:12.656')
plt.xlabel('Dates')
plt.ylabel('Counts')
ax.xaxis.set_tick_params(rotation=30, labelsize=10)

plt.show()

In [ ]:
# zoom into the plot, then pick an absolte time for an earthquake
print(starttime)
ta = starttime + 382000
print(ta)

In [ ]:
# compute FFT

# demean signal
y_demean = detrend(y,'constant')
# taper signal
taper_percentage = 1
taper = cosine_taper(npts,taper_percentage)
y_taper = y_demean * taper

# calculate FFT: amplitude of C(w), and phase of C(w), frequencies
fft_amp, fft_phase, f = wf_fft(y_taper,fNy)

# reconstruct the Fourier Transform C(w)
C = fft_amp*np.exp(1j*fft_phase)

In [ ]:
# Homework exercises start here!

#fig=plt.figure();
# YOUR PLOT COMMAND HERE
#cid = fig.canvas.mpl_connect('button_press_event', markp_min)